<a href="https://colab.research.google.com/github/geoskimoto/AWDB_SOAP_Request/blob/main/GetData_AWDB_SOAP_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import requests
import xml.dom.minidom as minidom
import json
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import datetime
url = "https://www.wcc.nrcs.usda.gov/awdbWebService/services?WSDL"


# Define Inputs
BeginDate = '03/01/2019'
EndDate = '08/01/2020'

# Create a dictionaries to store the data
headers = {'Content-type': 'text/soap'}
# current_dictionary = {}
 
# Define Web Service URL
URL = "https://wcc.sc.egov.usda.gov/awdbWebService/services?WSDL"

# Define Parameters for SOAP Elements (getData:current and getCentralTendencyData:normals)
SOAP_current = '''
<?xml version="1.0" encoding="UTF-8"?>
<SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/" xmlns:q0="http://www.wcc.nrcs.usda.gov/ns/awdbWebService" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <SOAP-ENV:Body>
    <q0:getData>
      <stationTriplets>302:OR:SNTL</stationTriplets>
      <elementCd>WTEQ</elementCd>   
      <ordinal>1</ordinal>
      <duration>DAILY</duration>
      <getFlags>false</getFlags>
      <beginDate>$BEGINDATE$</beginDate>
      <endDate>$ENDDATE$</endDate>
      <alwaysReturnDailyFeb29>true</alwaysReturnDailyFeb29>
    </q0:getData>
  </SOAP-ENV:Body>
</SOAP-ENV:Envelope>

'''.strip()

#other element codes: PREC, WTEQ (Water Equivalent/SWE)
 

# Post SOAP Elements to AWDB Web Service and process results - getData

SOAP_current = SOAP_current.replace("$BEGINDATE$", BeginDate)
SOAP_current = SOAP_current.replace("$ENDDATE$", EndDate)

response_current = requests.post(URL, data=SOAP_current, headers=headers)
xmldoc = minidom.parseString(response_current.text)

val_length = len(xmldoc.getElementsByTagName('values'))
data = pd.DataFrame([xmldoc.getElementsByTagName('values')[i].firstChild.data for i in range(0,val_length)])

date = datetime.datetime.strptime(BeginDate, "%m/%d/%Y").date() #.strftime("%Y,%m,%d")

Date = []
for i in range(val_length): 
  date += datetime.timedelta(days=1)
  Date.append(str(date))

# {str(xmldoc.getElementsByTagName('stationTriplet')[0].firstChild.data):{Date[j]:xmldoc.getElementsByTagName('values')[j].firstChild.data} for j in range(3)}

data['Date'] = Date
data.columns = ['WTEQ','Date']
# data.rename(columns={0:'SWE'}) #doesn't work for some reason, so renaming with just data.columns
data.set_index('Date', inplace=True)
data.tail(7)

,WTEQ
Date,
2020-07-27,0.0
2020-07-28,0.0
2020-07-29,0.0
2020-07-30,0.0
2020-07-31,0.0
2020-08-01,0.0
2020-08-02,0.0


In [8]:
data.head()

,WTEQ
Date,
2019-03-02,16.8
2019-03-03,16.9
2019-03-04,16.8
2019-03-05,16.8
2019-03-06,16.8


In [34]:
import requests
import xml.dom.minidom as minidom
import json
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import datetime

def SOAP_Call(stationtriplets, elementCD, begindate, enddate):

  # Create a dictionaries to store the data
  headers = {'Content-type': 'text/soap'}
  # current_dictionary = {}
  
  # Define Web Service URL
  URL = "https://wcc.sc.egov.usda.gov/awdbWebService/services?WSDL"

  # Define Parameters for SOAP Elements (getData:current and getCentralTendencyData:normals)
  SOAP_current = '''
  <?xml version="1.0" encoding="UTF-8"?>
  <SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/" xmlns:q0="http://www.wcc.nrcs.usda.gov/ns/awdbWebService" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
    <SOAP-ENV:Body>
      <q0:getData>
        <stationTriplets>$STATIONTRIPLETS$</stationTriplets>
        <elementCd>$ELEMENTCD$</elementCd>   
        <ordinal>1</ordinal>
        <duration>DAILY</duration>
        <getFlags>false</getFlags>
        <beginDate>$BEGINDATE$</beginDate>
        <endDate>$ENDDATE$</endDate>
        <alwaysReturnDailyFeb29>true</alwaysReturnDailyFeb29>
      </q0:getData>
    </SOAP-ENV:Body>
  </SOAP-ENV:Envelope>

  '''.strip()

  #other element codes: PREC, WTEQ (Water Equivalent/SWE)
  

  # Post SOAP Elements to AWDB Web Service and process results - getData
  SOAP_current = SOAP_current.replace("$ELEMENTCD$", elementCD)
  SOAP_current = SOAP_current.replace("$STATIONTRIPLETS$", stationtriplets)
  SOAP_current = SOAP_current.replace("$BEGINDATE$", begindate)
  SOAP_current = SOAP_current.replace("$ENDDATE$", enddate)

  response_current = requests.post(URL, data=SOAP_current, headers=headers)
  xmldoc = minidom.parseString(response_current.text)

  val_length = len(xmldoc.getElementsByTagName('values'))
  data = pd.DataFrame([xmldoc.getElementsByTagName('values')[i].firstChild.data for i in range(0,val_length)])

  date = datetime.datetime.strptime(BeginDate, "%d/%m/%Y").date() #.strftime("%Y,%m,%d")

  Date = []
  for i in range(val_length): 
    date += datetime.timedelta(days=1)
    Date.append(str(date))

  # {str(xmldoc.getElementsByTagName('stationTriplet')[0].firstChild.data):{Date[j]:xmldoc.getElementsByTagName('values')[j].firstChild.data} for j in range(3)}

  data['Date'] = Date
  data.columns = ['WTEQ','Date']
  # data.rename(columns={0:'SWE'}) #doesn't work for some reason, so renaming with just data.columns
  data.set_index('Date', inplace=True)
  return data
  # data.tail(7)

In [36]:
# 471:ID:SNTL
SOAP_Call(stationtriplets='471:ID:SNTL',elementCD='PREC',begindate='04/04/2019',enddate='01/08/2020')

,WTEQ
Date,
2019-01-04,25.1
2019-01-05,25.1
2019-01-06,25.2
2019-01-07,25.5
2019-01-08,25.6
...,...
2019-10-06,9.1
2019-10-07,9.1
2019-10-08,9.5
